Below we're using Keras, to train an NN with images of cats and dogs.

In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Dense, Lambda
from keras import backend as K
from keras.callbacks import *


def global_average_pooling(x):
    return K.mean(x, axis = (1, 2))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Lambda(global_average_pooling, 
              output_shape=global_average_pooling_shape))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(model.summary())

model.load_weights("weights/weights.50-0.47.h5")

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

checkpoint_path="weights/weights.{epoch:02d}-{val_loss:.2f}.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss',
                             verbose=0, save_best_only=False, save_weights_only=False, mode='auto')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    )

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=checkpoint)

# model.save_weights('weights.h5')



Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 15, 15, 64)      

125/125 [==============================] - 36s 285ms/step - loss: 0.1375 - accuracy: 0.9495 - val_loss: 0.7039 - val_accuracy: 0.8200
Epoch 46/50
125/125 [==============================] - 35s 278ms/step - loss: 0.1231 - accuracy: 0.9561 - val_loss: 1.5849 - val_accuracy: 0.7900
Epoch 47/50
125/125 [==============================] - 31s 249ms/step - loss: 0.1640 - accuracy: 0.9501 - val_loss: 0.6224 - val_accuracy: 0.8363
Epoch 48/50
125/125 [==============================] - 31s 245ms/step - loss: 0.1011 - accuracy: 0.9624 - val_loss: 0.7145 - val_accuracy: 0.8438
Epoch 49/50
125/125 [==============================] - 31s 247ms/step - loss: 0.1186 - accuracy: 0.9519 - val_loss: 0.8333 - val_accuracy: 0.8350
Epoch 50/50
125/125 [==============================] - 31s 246ms/step - loss: 0.1151 - accuracy: 0.9574 - val_loss: 0.7599 - val_accuracy: 0.8413
